In [1]:
#imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

cdat = pd.read_csv('Texas_Counties.csv', usecols=['County']) #all 254 counties in TX
counties = cdat['County'].to_list()

non_ercot_counties = ['El Paso','Hudspeth','Gaines','Terry','Yoakum','Cochran','Hockley','Lubbock','Bailey',
                      'Lamb','Hartley','Dallam','Moore','Sherman','Hansford','Hutchinson','Ochiltree','Lipscomb', 
                      'Hemphill','Bowie','Morris','Cass','Camp','Marion','Upshur','Gregg','Harrison','Panola',
                      'Shelby','San Augustine','Sabine','Trinity','Polk','Tyler','Jasper','Newton','San Jacinto',
                      'Hardin','Liberty','Orange','Jefferson']
remove_counties = [count + ' County' for count in non_ercot_counties]

for count in remove_counties:
    counties.remove(count)

print(len(counties))

213


ANALYZING DATA

In [20]:
#coal data - if doing leave one out cross test, only use 2 years of coal data
coal = pd.read_csv('ERCOT/Coal2019.csv',usecols=['Coal']).to_numpy().flatten()
coalhours = np.hstack((coal,coal,coal))

In [ ]:
#Figure 5 (and supplement)
days = np.array([31,28,31,30,31,30,31,31,30,31,30,31]); halfhours = days*48; hsums = []
for month in range(12):
    hsums.append(sum(halfhours[:month]))
hsums.append(17520)
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

outdic = {}
for m in range(12):
    avg = np.empty(48)
    month = coalhours[hsums[m]:hsums[m+1]]
    for hh in range(48):
        avg[hh] = sum(month[hh::48]) / days[m]
    outdic[months[m]] = avg
pd.DataFrame(data=outdic).to_csv('CoalDays.csv')

In [5]:
#import formatted power data and stack for 3 years. Analyzing output within each county
cnum = len(counties)
solarcsvs = ['Solar/Solar2009.csv','Solar/Solar2010.csv','Solar/Solar2011.csv']
solartraincsvs = solarcsvs
solartestcsvs = []
solarpower = np.empty((cnum,0))
solarpowertest = np.empty((cnum,0))

#test/train split the csvs if you want to do leave one out cross-test analysis
for solarcsv in solartraincsvs:
    pdat = pd.read_csv(solarcsv, usecols=counties)
    solaryear = pdat[counties[0]].to_numpy() #0th
    for i in range(cnum-1): #1th onwards
        solaryear = np.vstack((solaryear,pdat[counties[i+1]].to_numpy()))
    solarpower = np.concatenate((solarpower,solaryear),axis=1) #div by 2 to convert from MW (power) to MWh (energy output)

for solarcsv in solartestcsvs:
    pdat = pd.read_csv(solarcsv, usecols=counties)
    solaryear = pdat[counties[0]].to_numpy() #0th
    for i in range(cnum-1): #1th onwards
        solaryear = np.vstack((solaryear,pdat[counties[i+1]].to_numpy()))
    solarpowertest = np.concatenate((solarpowertest,solaryear),axis=1)

#NEED TO PUT IN WIND2011.csv
windcsvs = ['Wind/Wind2009.csv','Wind/Wind2010.csv','Wind/Wind2010.csv']
windtraincsvs = windcsvs
windtestcsvs = []
windpower = np.empty((cnum,0))
windpowertest = np.empty((cnum,0))

for windcsv in windtraincsvs:
    pdat = pd.read_csv(windcsv, usecols=counties)
    windyear = pdat[counties[0]].to_numpy() #0th
    for i in range(cnum-1): #1th onwards
        windyear = np.vstack((windyear,pdat[counties[i+1]].to_numpy()))
    windpower = np.concatenate((windpower,windyear),axis=1)
    
for windcsv in windtestcsvs:
    pdat = pd.read_csv(windcsv, usecols=windcounts)
    windyear = pdat[counties[0]].to_numpy() #0th
    for i in range(cnum-1): #1th onwards
        windyear = np.vstack((windyear,pdat[counties[i+1]].to_numpy()))
    windpowertest = np.concatenate((windpowertest,windyear),axis=1)
    
solarpower = np.transpose(solarpower)
windpower = np.transpose(windpower)
solarpowertest = np.transpose(solarpowertest)
windpowertest = np.transpose(windpowertest)

In [ ]:
#Figure 6 - this csv file can be used to make the map
sp = sum(solarpower) / 52560
wp = sum(windpower) / 52560
outdic = {'County': counties, 'Solar': 2*sp, 'Wind': 2*wp}
pd.DataFrame(data=outdic).to_csv('AverageCF_County.csv')

RUNNING OPTIMIZATION

In [16]:
#USING GIS REPORT

#import GIS site data
gisreport = 'ERCOT/Ercot_GIS_Report_June.xlsx'
pdat = pd.read_excel(gisreport, sheet_name='Project Details', usecols=['GINR Study Phase','County','Fuel','Capacity (MW)'])

solarp = pdat.loc[pdat['Fuel']=="SOL"]
solarp = solarp.loc[solarp['Capacity (MW)'] > 0] #some caps 0 - don't need to remove technically but y not

ogsolarcounts = solarp['County'].to_list()
solarcounts = [count + ' County' for count in ogsolarcounts]
solarcaps = solarp['Capacity (MW)'].to_numpy()
solarcnum = len(solarcounts)

windp = pdat.loc[pdat['Fuel']=="WIN"]
windp = windp.loc[windp['Capacity (MW)'] > 0] 

ogwindcounts = windp['County'].to_list()
windcounts = [count + ' County' for count in ogwindcounts]
windcaps = windp['Capacity (MW)'].to_numpy()
windcnum = len(windcounts)

solarnames = [solarcounts[i] + ' Solar: ' + str(solarcaps[i]) for i in range(solarcnum)]
windnames = [windcounts[i] + ' Wind: ' + str(windcaps[i]) for i in range(windcnum)] 

####for checking if things ran correctly
print(solarcnum)
print(windcnum)

print(sum(solarcaps))
print(sum(windcaps))
print(max(solarcaps))
print(max(windcaps))


262
108
58014.529999999984
24588.49
610.0
630.0


In [ ]:
#Figure 4 - Study Phase info
#import GIS site data
gisreport = 'ERCOT/Ercot_GIS_Report_June.xlsx'
pdat = pd.read_excel(gisreport, sheet_name='Project Details', usecols=['GINR Study Phase','County','Fuel','Capacity (MW)'])

solarp = pdat.loc[pdat['Fuel']=="SOL"]
solarp = solarp.loc[solarp['Capacity (MW)'] > 0] #some caps 0 - don't need to remove technically but y not
solarcaps = solarp['Capacity (MW)'].to_numpy()
sphase = solarp['GINR Study Phase'].to_list()

windp = pdat.loc[pdat['Fuel']=="WIN"]
windp = windp.loc[windp['Capacity (MW)'] > 0] 
windcaps = windp['Capacity (MW)'].to_numpy()
wphase = windp['GINR Study Phase'].to_list()

spd = defaultdict(int)
wpd = defaultdict(int)

for i in range(len(solarcaps)):
    phase = sphase[i]
    cap = solarcaps[i]
    spd[phase] += float(cap)
for i in range(len(windcaps)):
    phase = wphase[i]
    cap = windcaps[i]
    wpd[phase] += float(cap)
    
pd.DataFrame(data=[spd,wpd]).to_excel('StudyPhaseOut.xlsx')

In [17]:
#import formatted power data and scale up by site capacity, put in matrix form
solarcsvs = ['Solar/Solar2009.csv','Solar/Solar2010.csv','Solar/Solar2011.csv']
solartraincsvs = solarcsvs
solartestcsvs = []
solarpower = np.empty((solarcnum,0))
solarpowertest = np.empty((solarcnum,0))
hl = 0.5 #working with half hours so hour-length = 0.5

#test/train split the csvs if you want to do leave one out cross-test analysis
for solarcsv in solartraincsvs:
    pdat = pd.read_csv(solarcsv, usecols=solarcounts)
    solaryear = pdat[solarcounts[0]].to_numpy() * solarcaps[0] * hl #0th
    for i in range(solarcnum-1): #1th onwards
        solaryear = np.vstack((solaryear,pdat[solarcounts[i+1]].to_numpy() * solarcaps[i+1] * hl))
    solarpower = np.concatenate((solarpower,solaryear),axis=1) #div by 2 to convert from MW (power) to MWh (energy output)

for solarcsv in solartestcsvs:
    pdat = pd.read_csv(solarcsv, usecols=solarcounts)
    solaryear = pdat[solarcounts[0]].to_numpy() * solarcaps[0] * hl #0th
    for i in range(solarcnum-1): #1th onwards
        solaryear = np.vstack((solaryear,pdat[solarcounts[i+1]].to_numpy() * solarcaps[i+1] * hl))
    solarpowertest = np.concatenate((solarpowertest,solaryear),axis=1)

#NEED TO PUT IN WIND2011.csv
windcsvs = ['Wind/Wind2009.csv','Wind/Wind2010.csv','Wind/Wind2010.csv']
windtraincsvs = windcsvs
windtestcsvs = []
windpower = np.empty((windcnum,0))
windpowertest = np.empty((windcnum,0))

for windcsv in windtraincsvs:
    pdat = pd.read_csv(windcsv, usecols=windcounts)
    windyear = pdat[windcounts[0]].to_numpy() * windcaps[0] * hl #0th
    for i in range(windcnum-1): #1th onwards
        windyear = np.vstack((windyear,pdat[windcounts[i+1]].to_numpy() * windcaps[i+1] * hl))
    windpower = np.concatenate((windpower,windyear),axis=1)
    
for windcsv in windtestcsvs:
    pdat = pd.read_csv(windcsv, usecols=windcounts)
    windyear = pdat[windcounts[0]].to_numpy() * windcaps[0] * hl #0th
    for i in range(windcnum-1): #1th onwards
        windyear = np.vstack((windyear,pdat[windcounts[i+1]].to_numpy() * windcaps[i+1] * hl))
    windpowertest = np.concatenate((windpowertest,windyear),axis=1)
    
solarpower = np.transpose(solarpower)
windpower = np.transpose(windpower)
solarpowertest = np.transpose(solarpowertest)
windpowertest = np.transpose(windpowertest)

In [ ]:
#Figure 7 - Monthly regional capacity factor --> first get regional info then use to make monthly graphs
simpregdict = pd.read_csv('CountyToRegion.csv')
regions = np.array(['NORTH', 'NCENT', 'EAST', 'COAST', 'SOUTH', 'SCENT', 'WEST', 'FWEST']) #easier if numpy array

y = 17520
n = 3*y
solarregout = dict.fromkeys(regions,np.zeros(y)) #every half-hour
solarregcap = dict.fromkeys(regions,0)
windregout = dict.fromkeys(regions,np.zeros(y))
windregcap = dict.fromkeys(regions,0)

#if you want to analyze all GIS sites use this
solarout = counties
windout = counties

for site in solarout:
    i = solarnames.index(site)
    sitepower = solarpower[:,i]
    cap = solarcaps[i]
    county = solarcounts[i]
    reg = simpregdict[county][0]
    avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
    solarregout[reg] = solarregout[reg] + avpower #((sitepower[::2] + sitepower[1::2]) / 1) #divide by 2 b/c want avg power (if MWh don't)
    solarregcap[reg] = solarregcap[reg] + cap
    
for site in windout:
    i = windnames.index(site)
    sitepower = windpower[:,i]
    cap = windcaps[i]
    county = windcounts[i]
    reg = simpregdict[county][0]
    avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
    windregout[reg] = windregout[reg] + avpower
    windregcap[reg] = windregcap[reg] + cap
    
solarregcf = {}
windregcf = {}
regcap = {}
for reg in regions:
    solarregcf[reg] = 2*solarregout[reg] / solarregcap[reg]
    windregcf[reg] = 2*windregout[reg] / windregcap[reg]
    regcap[reg] = [solarregcap[reg],windregcap[reg]]    
#pd.DataFrame(data=regcap).to_csv('RegOut.csv')

#Figure 7 - Monthly regional capacity factor
smregs = {}
wmregs = {}
cmregs = {}

targetregs = ['NORTH','COAST','SOUTH','FWEST']
for month in range(12):
    outdic = {}
    for reg in targetregs:
        solaryear = solarregcf[reg]
        windyear = windregcf[reg]
        solarmonth = solaryear[hsums[month]:hsums[month+1]]
        windmonth = windyear[hsums[month]:hsums[month+1]]
        solarday = np.empty(48)
        windday = np.empty(48)
        for hh in range(48):
            solarday[hh] = sum(solarmonth[hh::48]) / days[month]
            windday[hh] = sum(windmonth[hh::48]) / days[month]
        outdic[reg + 'Solar'] = solarday
        outdic[reg + 'Wind'] = windday
    pd.DataFrame(data=outdic).to_csv(months[month]+'.csv') #this will create 12 csvs for each month, combine into one sheet after
        
# pd.DataFrame(data=smregs).to_csv('SolarMonthlyCF.csv')
# pd.DataFrame(data=wmregs).to_csv('WindMonthlyCF.csv')
# pd.DataFrame(data=cmregs).to_csv('CoalMonthlyCF.csv')

In [10]:
#cost data
sc = 92.50*1000 # LCOE $ / MW-Cap 
solarcost = sc*solarcaps
wc = 118.48*1000 # LCOE $ / MW-Cap 
windcost = wc*windcaps
#coalcost = 111.67 # $ / MWh

In [18]:
fuel = 'ERCOT/Fuel2019.xlsx'
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
xls = pd.ExcelFile(fuel)

coalhours = np.empty(0)

for i in range(12):
    dat = pd.read_excel(xls, months[i])
    coal = dat.loc[dat['Fuel']=="Coal"]
    coal = coal[coal.columns[4::2]].to_numpy() + coal[coal.columns[5::2]].to_numpy() #every half hour
    coal = np.reshape(coal,np.prod(coal.shape))
    coalhours = np.concatenate((coalhours,coal))
coalhours = np.nan_to_num(coalhours)
coalhours = np.hstack((coalhours,coalhours,coalhours))

In [24]:
#Exploratory analysis
coaltotal = sum(coalhours)
h = len(coalhours)

#for continouous
#s_maxcap = 610; w_maxcap = 630
#spv = s_maxcap*np.sum(solarpower, axis=1); wpv = w_maxcap*np.sum(windpower, axis=1);
spv = np.sum(solarpower, axis=1); wpv = np.sum(windpower, axis=1);
allpower = spv + wpv

sd = coalhours - spv
wd = coalhours - wpv
dif = coalhours - allpower
sinf = sum(sd[sd > 0])
winf = sum(wd[wd > 0])
infeasibility = sum(dif[dif > 0]) 
infhours = sum(dif > 0)

print("Model timespan:", h / (48*365), "years")
print("There are a total of", sum(solarcaps), "MW of solar capacity in GIS queue")
print("There are a total of", sum(windcaps), "MW of wind capacity in GIS queue")
print("Building all solar sites (and no wind) can cover:", 100*(1 - (sinf / coaltotal)), "% of coal output")
print("Building all wind sites (and no solar) can cover:", 100*(1 - (winf / coaltotal)), "% of coal output")
print("Building all wind and solar sites can cover:", 100*(1 - (infeasibility / coaltotal)), "% of coal output")
print("Building all sites can cover coal output on:", 100*(1 - (infhours / h)), "% of half-hours")
print("Building all sites would cost: $", (sum(solarcost) + sum(windcost)) / 10**9, "billion")

print(coalhours.shape)
slackcap = (max(coalhours))
slacknames = ['Slack:' + str(i) for i in range(h)] #for naming slack variables in model

Model timespan: 3.0 years
There are a total of 58014.529999999984 MW of solar capacity in GIS queue
There are a total of 24588.49 MW of wind capacity in GIS queue
Building all solar sites (and no wind) can cover: 52.53644870668295 % of coal output
Building all wind sites (and no solar) can cover: 78.0965847877315 % of coal output
Building all wind and solar sites can cover: 96.18286520350973 % of coal output
Building all sites can cover coal output on: 88.90981735159818 % of half-hours
Building all sites would cost: $ 8.2795883202 billion
(52560,)


In [7]:
#setting up model and solving
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("coal")

#vars
s = m.addMVar(solarcnum, lb=0,ub=1, vtype=GRB.BINARY, name=solarnames) #indicator var for if site is built
w = m.addMVar(windcnum, lb=0,ub=1, vtype=GRB.BINARY, name=windnames)
slack = m.addMVar(h, lb=0, ub=slackcap, vtype=GRB.CONTINUOUS, name=slacknames)

#for continuous model - upper_bound for solar/wind is max capacity for each fuel type in June 2020 GIS report
#s = m.addMVar(solarcnum, lb=0,ub=610, vtype=GRB.CONTINUOUS, name=solarnames)
#w = m.addMVar(windcnum, lb=0,ub=630, vtype=GRB.CONTINUOUS, name=windnames)

#objective
m.setObjective((solarcost @ s) + (windcost @ w), GRB.MINIMIZE)

#constraints
for hour in range(h):
    m.addConstr((solarpower[hour,:] @ s) + (windpower[hour,:] @ w) >= coalhours[hour] - slack[hour])
m.addConstr(sum(slack) <= 0.1*coaltotal) #allow % of total coal demand to be slacked


#optimizing
m.Params.MIPGap = 10**(-6)
m.optimize()

Using license file /Users/richard.morse/gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-22
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 52561 rows, 52930 columns and 19393555 nonzeros
Model fingerprint: 0x71ced68e
Variable types: 52560 continuous, 370 integer (370 binary)
Coefficient statistics:
  Matrix range     [3e-06, 3e+02]
  Objective range  [2e+05, 7e+07]
  Bounds range     [1e+00, 7e+03]
  RHS range        [1e+03, 2e+07]
Found heuristic solution: objective 7.909265e+09
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve removed 0 rows and 28 columns (presolve time = 48s) ...
Presolve removed 0 rows and 28 columns (presolve time = 50s) ...
Presolve removed 5303 rows and 5331 columns (presolve time = 64s) ...
Presolve removed 5303 rows and 5331 columns (presolve time = 65s) ...
Presolve removed 5303 rows and 5333 columns (pres

H    0     0                    2.812361e+09 2.8110e+09  0.05%     -  527s
     0     0 2.8110e+09    0    8 2.8124e+09 2.8110e+09  0.05%     - 3009s
     0     0 2.8110e+09    0   10 2.8124e+09 2.8110e+09  0.05%     - 3021s
     0     0 2.8110e+09    0   11 2.8124e+09 2.8110e+09  0.05%     - 3027s
     0     0 2.8110e+09    0   10 2.8124e+09 2.8110e+09  0.05%     - 3031s
     0     0 2.8110e+09    0   11 2.8124e+09 2.8110e+09  0.05%     - 3035s
     0     0 2.8110e+09    0   12 2.8124e+09 2.8110e+09  0.05%     - 3047s
     0     0 2.8110e+09    0   11 2.8124e+09 2.8110e+09  0.05%     - 3054s
     0     0 2.8110e+09    0   11 2.8124e+09 2.8110e+09  0.05%     - 3057s
     0     0 2.8110e+09    0   13 2.8124e+09 2.8110e+09  0.05%     - 3082s
     0     0 2.8110e+09    0   12 2.8124e+09 2.8110e+09  0.05%     - 3088s
     0     0 2.8110e+09    0   15 2.8124e+09 2.8110e+09  0.05%     - 3113s
     0     0 2.8110e+09    0   14 2.8124e+09 2.8110e+09  0.05%     - 3120s
     0     0 2.8110e+09  

INTERROGATING MODEL RESULTS

In [10]:
#investigating model and reporting results
#h = len(coalhours)
slackedhours = 0 #how many hours did we not meet (significantly i.e. more than 1% off)
slacksum = 0 #how much total load did we not meet
powersum = 0
slackout = []
solarout = []
windout = []
smallsites = 0

for v in m.getVars():
    if v.x > 0.01:
        name = v.varName
        if 'Wind' in name:
            windout.append(name)
            if v.x < 1:
                smallsites += 1
            
        elif 'Solar' in name:
            solarout.append(name)
            if v.x < 1:
                smallsites += 1
        elif 'Slack' in name:
            slackedhours += 1
            slacksum += v.x
            [nada,hour] = name.split(':')
            slackout.append(int(hour))
        else:
            powersum += v.x

sint = [round(val) for val in s.x] 
wint = [round(val) for val in w.x]
sitecost = np.matmul(solarcost,sint) + np.matmul(windcost,wint)
#coaltotalcost = sum(slack.x)*coalcost
solarpowerout = np.matmul(solarpower,sint)
windpowerout = np.matmul(windpower,wint)
powerout = solarpowerout + windpowerout
    
print('Solar sites used: %g' % len(solarout))
print('Wind sites used: %g' % len(windout))
print('Percent of Hours slacked: %g' % (slackedhours/h))
print('Percent of Load slacked: %g' % (slacksum / coaltotal))
print('Build Cost (Billion) %g' % (sitecost / 10**9))

#saving site output
#pd.DataFrame(data={'Solar': solarpowerout, 'Wind': windpowerout}).to_csv('ModelOut/Power90.csv')

#saving solver info
#m.write('out.mst')
#m.write('out.attr')
#m.write('out.mps')

NameError: name 'm' is not defined

In [ ]:
#saving model chosen sites info - Figure 9
solarmodel = defaultdict(float)
windmodel = defaultdict(float)
for site in solarout:
    [county,cap] = site.split(' Solar: ')
    solarmodel[county] += float(cap)
for site in windout:
    [county,cap] = site.split(' Wind: ')
    windmodel[county] += float(cap)

#combined all these sheets into Sites90.xlsx spreadsheet after running this cell
pd.DataFrame(data=solarmodel,index=['Capacities']).to_csv('ModelOut/SolarOutSites.csv')
pd.DataFrame(data=windmodel,index=['Capacities']).to_csv('ModelOut/WindOutSites.csv')
pd.DataFrame(data={'Sites': solarout}).to_csv('ModelOut/SitesS.csv')
pd.DataFrame(data={'Sites': windout}).to_csv('ModelOut/SitesW.csv')

In [ ]:
#TABLE 4 - Testing generalizability to other years (only use this if doing train/test split)
powertest = np.matmul(solarpowertest,sint) + np.matmul(windpowertest,wint)
slacktest = 0 
hourtest = 0
dif = np.transpose(coalhours[0:17520]) - powertest

for i in range(17520):
    if dif[i] > 0:
        slacktest += dif[i]
        hourtest += 1
print(1-(slacktest / (coaltotal/2)))
print(1-(hourtest / 17520))

In [ ]:
#Distribution - NOT USED
coalcap = 14225
#coal = pd.read_csv('Coal2019.csv',usecols=['Coal']).to_numpy().flatten()
#coal = np.flip(np.sort(2*coal))
#coalcf = coal / coalcap

pdat = pd.read_csv('ModelOut/Power90.csv')
solarp = 2*pdat['Solar'].to_numpy()
windp = 2*pdat['Wind'].to_numpy()
power = solarp + windp

# y = 17520
# syear1 = solarp[0:y]
# syear2 = solarp[y:2*y]
# syear3 = solarp[2*y:3*y]
# wyear1 = windp[0:y]
# wyear2 = windp[y:2*y]
# wyear3 = windp[2*y:3*y]
# ayear1 = power[0:y]
# ayear2 = power[y:2*y]
# ayear3 = power[2*y:3*y]

solarcap = 11706.08
windcap = 16611.35
totalcap = solarcap + windcap

# spd1 = np.flip(np.sort(syear1 / spoweravg))
# spd2 = np.flip(np.sort(syear2 / spoweravg))
# spd3 = np.flip(np.sort(syear3 / spoweravg))
# wpd1 = np.flip(np.sort(wyear1 / wpoweravg))
# wpd2 = np.flip(np.sort(wyear2 / wpoweravg))
# wpd3 = np.flip(np.sort(wyear3 / wpoweravg))
# apd1 = np.flip(np.sort(ayear1 / apoweravg))
# apd2 = np.flip(np.sort(ayear2 / apoweravg))
# apd3 = np.flip(np.sort(ayear3 / apoweravg))

solarp = np.flip(np.sort(solarp / solarcap))
windp = np.flip(np.sort(windp / windcap))
power = np.flip(np.sort(power / totalcap))

#yeardic = {'Solar 2009': spd1, 'Solar 2010': spd2, 'Solar 2011': spd3, 'Wind 2009': wpd1, 'Wind 2010': wpd2, 'Wind 2011': wpd3, 'Both 2009': apd1, 'Both 2010': apd2, 'Both 2011': apd3}
alldic = {'Solar': solarp, 'Wind': windp, 'Both': power}

#pd.DataFrame(data={'Coal Dist': coalyear}).to_csv('CoalDistrYear.csv')
#pd.DataFrame(data={'Coal Dist': coalall}).to_csv('CoalDistrExtended.csv')
#pd.DataFrame(data=yeardic).to_csv('ModelDistrYears.csv')
pd.DataFrame(data=alldic).to_csv('ModelDistr.csv')


In [ ]:
#Monthly average Capacity Factor graph for each fuel type - NOT USED

solarpoweragg = np.sum(solarpower,axis=1) #/ 213 #213 counties
windpoweragg = np.sum(windpower,axis=1) #/ 213

coal = 'Coal2019.csv'
coalhours = pd.read_csv(coal,usecols=['Coal']).to_numpy()

solaryear = np.empty(17520)
windyear = np.empty(17520)
for i in range(17520):
    solaryear[i] = sum(solarpoweragg[i::17520])/3
    windyear[i] = sum(windpoweragg[i::17520])/3

solaryear = 2*solaryear / sum(solarcaps)
windyear = 2*windyear / sum(windcaps)
coalhours = 2*coalhours / 15065
days = [31,28,31,30,31,30,31,31,30,31,30,31]
d=0

sma = np.empty(12); wma = np.empty(12); cma = np.empty(12);
for month in range(12):
        sma[month] = sum(solaryear[(d*48):((d+days[month])*48)]) / (days[month]*48)
        wma[month] = sum(windyear[(d*48):((d+days[month])*48)]) / (days[month]*48)
        cma[month] = sum(coalhours[(d*48):((d+days[month])*48)]) / (days[month]*48) #coal max = 6950
        d += days[month]
        
# pd.DataFrame(data=sma).to_csv('SolarMonthlyCF.csv')
# pd.DataFrame(data=wma).to_csv('WindMonthlyCF.csv')
# pd.DataFrame(data=cma).to_csv('CoalMonthlyCF.csv')

In [ ]:
loadzones = ['HB_NORTH', 'HB_SOUTH', 'HB_WEST']
weathertoload = {'NORTH': 'HB_WEST', 'NCENT': 'HB_NORTH', 'EAST': 'HB_NORTH', 'COAST': 'HB_SOUTH', 'SOUTH': 'HB_SOUTH', 'SCENT': 'HB_SOUTH', 'WEST': 'HB_WEST', 'FWEST': 'HB_WEST'}

In [ ]:
#getting regional info
y = 17520
n = 3*y
solarregout = dict.fromkeys(regions,np.zeros(y)) #every half-hour
solarregcap = dict.fromkeys(regions,0)
windregout = dict.fromkeys(regions,np.zeros(y))
windregcap = dict.fromkeys(regions,0)

#if you want to analyze only 90% model sites use this
pdat = pd.read_excel('Sites90.xlsx',sheet_name='SitesLong')
solarout = pdat['Solar'].dropna().to_list()
windout = pdat['Wind'].dropna().to_list()

#to compare regional load to power
# solarloadout = dict.fromkeys(loadzones,np.zeros(y))
# windloadout = dict.fromkeys(loadzones,np.zeros(y))

#to look at revenue,cost,profit
solarprofit = np.zeros(262)
windprofit = np.zeros(108)

for site in solarout:
    i = solarnames.index(site)
    sitepower = solarpower[:,i]
    cap = solarcaps[i]
    county = solarcounts[i]
    reg = simpregdict[county][0]
    avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
    solarregout[reg] = solarregout[reg] + avpower #((sitepower[::2] + sitepower[1::2]) / 1) #divide by 2 b/c want avg power (if MWh don't)
    solarregcap[reg] = solarregcap[reg] + cap
    
    #if you want to compare regional load to power
#     avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
#     cost = solarcost[i]
#     loadreg = weathertoload[reg]
#     solarloadout[loadreg] = solarloadout[loadreg] + avpower
    #if you want to look at revenue,cost,profit
#     rev = np.matmul(energyprice[loadreg],avpower)
#     solarprofit[i] = (rev - cost) #annual
    
for site in windout:
    i = windnames.index(site)
    sitepower = windpower[:,i]
    cap = windcaps[i]
    county = windcounts[i]
    reg = simpregdict[county][0]
    avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
    windregout[reg] = windregout[reg] + avpower
    windregcap[reg] = windregcap[reg] + cap
    
    #if you want to compare regional load to power
#     avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
#     cost = windcost[i]
#     loadreg = weathertoload[reg]
#     windloadout[loadreg] = windloadout[loadreg] + avpower
    #if you want to look at revenue,cost,profit
#     rev = np.matmul(energyprice[loadreg],avpower)
#     windprofit[i] = (rev - cost) #annual
    
solarregcf = {}
windregcf = {}
regcap = {}
for reg in regions:
    solarregcf[reg] = 2*solarregout[reg] / solarregcap[reg]
    windregcf[reg] = 2*windregout[reg] / windregcap[reg]
    regcap[reg] = [solarregcap[reg],windregcap[reg]]    
#pd.DataFrame(data=regcap).to_csv('RegOut.csv')

In [ ]:
#which sites are the most efficient (cost per MW), what range of costs do we have (Figure 10)
coaltotal = sum(coalhours)
coalgwh = coaltotal / (3*10**3)
pdat = pd.read_excel('Sites90.xlsx',sheet_name='SitesLong')
solarout = pdat['Solar'].dropna().to_list()
windout = pdat['Wind'].dropna().to_list()
sitesout = pdat['Solar'].dropna().to_list() + pdat['Wind'].to_list()

#2.81 billion

ssum = sum(solarpower) / 3
wsum = sum(windpower) / 3
seff = solarcost / ssum
weff = windcost / wsum
#seff = solarprofit / (10**6)
#weff = windprofit / (10**6)
twine = []

for i in range(windcnum):
    #twine.append((seff[i],ssum[i],solarnames[i]))
    #twine.append((weff[i],wsum[i],windnames[i]))
    twine.append((seff[i],solarcost[i],i,solarnames[i], ssum[i]))
    twine.append((weff[i],windcost[i],i,windnames[i], wsum[i]))
for i in range(windcnum,solarcnum):
    #twine.append((seff[i],ssum[i],solarnames[i]))
    twine.append((seff[i],solarcost[i],i,solarnames[i], ssum[i]))

twine.sort()
#twine.reverse() #need to reverse for falling profitability
order = []
efforder = []
sumorder = []

fcolor = []
totalcost = 0
powerpack = np.zeros(52560)
for i in range(windcnum+solarcnum):
    name = twine[i][3]
    idx = twine[i][2]
    cost = twine[i][1]
    order.append(name)
    efforder.append(twine[i][0])
    sumorder.append(twine[i][4] / 10**3)
    if 'Solar:' in name:
        fcolor.append('bisque')
        powerpack += solarpower[:,idx]
    else:
        fcolor.append('lightsteelblue')
        powerpack += windpower[:,idx]

        
for site in sitesout:
    i = order.index(site)
    if fcolor[i] == 'bisque':
        fcolor[i] = 'darkorange'
    elif fcolor[i] == 'lightsteelblue':
        fcolor[i] = 'blue'

sp = 0; sc = 0;
wp = 0; wc = 0;

for site in solarout:
    i = solarnames.index(site)
    sp += ssum[i]
    sc += solarcost[i]
for site in windout:
    i = windnames.index(site)
    wp += wsum[i]
    wc += windcost[i]
#print(sp,sc,wp,wc)
print((sc+wc) / (sp+wp))

# popi = []        
# for i in range(0,len(sitesout)-1):
#     n = i+1
#     count = order[i].split(':')[0]
#     sc = sumorder[i]
#     while count == order[n].split(':')[0]:
#         sc += sumorder[n]
#         popi.append(n)
#         n = n+1
#     sumorder[i] = sc
        
# for i in popi:
#     order.pop(i); efforder.pop(i); sumorder.pop(i); fcolor.pop(i);

xs = np.zeros(solarcnum+windcnum)
for i in range(1,solarcnum+windcnum):
    xs[i] = xs[i-1] + sumorder[i-1]
    if abs(efforder[i]) < 0.2:
        efforder[i] += (efforder[i] / (2*abs(efforder[i])))
        print(order[i],efforder[i])

srange = [min(seff),max(seff)]
wrange = [min(weff),max(weff)]
print('Solar Price Per MWh Range: %s' % srange)
print('Wind Price Per MWh Range: %s' % wrange)

# for i in range(windcnum):
#     sorder.append(stwine[i][1])
#     worder.append(wtwine[i][1])
# for i in range(windcnum,solarcnum):
#     sorder.append(stwine[i][1])

# sorder = list(dict.fromkeys(sorder)) #remove repeats
# worder = list(dict.fromkeys(worder))
# sorder = [count + ' Solar' for count in sorder]
# worder = [count + ' Wind' for count in worder]

#basically want to see whether its taking greedy 'best' sites or not
# sg = defaultdict(float)
# wg = defaultdict(float)


# for count in solarout:
#     [x,y] = count.split(':')
#     i = sorder.index(x)
#     sg[i+1] += float(y)
# for count in windout:
#     [x,y] = count.split(':')
#     i = worder.index(x)
#     wg[i+1] += float(y)

# sg = dict(sorted(sg.items()))
# wg = dict(sorted(wg.items()))

# pd.DataFrame(data=[sg,wg]).to_csv('Eff90Full.csv')

In [ ]:
#Figure 10 - efficiency of used and left out sites
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter
import math

fig,ax = plt.subplots(1, figsize=(10,5))
#plt.grid(zorder=0); ax.set_axisbelow(True)
plt.bar(x=xs, height=efforder, width=sumorder, align='edge', color=fcolor)
plt.axvline(x=coalgwh,linestyle='--',color='black'); plt.text(coalgwh+1000,40,'Coal GWh')
plt.xlabel('Annual Energy Supplied (GWh)'); plt.ylabel('Levelized Cost ($ / MWh)');
ax.set_xlim((0,xs[-1]+5000)); 
ax.xaxis.set_major_locator(MultipleLocator(25000)); 
#ax.xaxis.set_minor_locator(MultipleLocator(25000))
ax.get_xaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))

ax.yaxis.set_minor_locator(MultipleLocator(5))
ax.spines['bottom'].set_color('gray'); ax.spines['top'].set_color('white');
ax.spines['right'].set_color('white'); ax.spines['left'].set_color('gray');

lcolors = {'Wind: Selected':'blue', 'Wind: Unselected':'lightsteelblue','Solar: Selected':'darkorange', 'Solar: Unselected':'bisque'}   
lecolors = {'Wind: Selected':'blue', 'Wind: Unselected':'lightsteelblue','Solar: Selected':'darkorange', 'Solar: Unselected':'bisque'} 
labels = list(lcolors.keys())
handles = [plt.Rectangle((0,0),1,1, facecolor=lcolors[label], edgecolor=lecolors[label]) for label in labels]
#plt.legend(handles, labels)
ax.legend(handles, labels, bbox_to_anchor=(0.21, 1.05))
plt.tight_layout()

plt.savefig('Site_Selection_Bar_MarginalCost', dpi=300)

In [ ]:
#fuel mix info (Figure 11)

#fuel = 'Fuel2011.xls' 
#months = ['Jan11','Feb11','Mar11','Apr11','May11','Jun11','Jul11','Aug11','Sep11','Oct11','Nov11','Dec11']
fuel = 'Fuel2019.xlsx'
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
xls = pd.ExcelFile(fuel)

biomass = np.empty(0)
coal = np.empty(0)
gas = np.empty(0)
gascc = np.empty(0)
hydro = np.empty(0)
nuclear = np.empty(0)
other = np.empty(0)
solar = np.empty(0)
wind = np.empty(0)



for i in range(12):
    dat = pd.read_excel(xls, months[i])
    
    biomassyear = dat.loc[dat['Fuel']=="Biomass"]
    biomassyear = biomassyear[biomassyear.columns[4::2]].to_numpy() + biomassyear[biomassyear.columns[5::2]].to_numpy()
    biomassyear = np.reshape(biomassyear,np.prod(biomassyear.shape))
    biomass = np.concatenate((biomass,biomassyear))
    
    coalyear = dat.loc[dat['Fuel']=="Coal"]
    coalyear = coalyear[coalyear.columns[4::2]].to_numpy() + coalyear[coalyear.columns[5::2]].to_numpy()
    coalyear = np.reshape(coalyear,np.prod(coalyear.shape))
    coal = np.concatenate((coal,coalyear))
    
    gasyear = dat.loc[dat['Fuel']=="Gas"]
    gasyear = gasyear[gasyear.columns[4::2]].to_numpy() + gasyear[gasyear.columns[5::2]].to_numpy()
    gasyear = np.reshape(gasyear,np.prod(gasyear.shape))
    gas = np.concatenate((gas,gasyear))

    gasccyear = dat.loc[dat['Fuel']=="Gas-CC"]
    gasccyear = gasccyear[gasccyear.columns[4::2]].to_numpy() + gasccyear[gasccyear.columns[5::2]].to_numpy()
    gasccyear = np.reshape(gasccyear,np.prod(gasccyear.shape))
    gascc = np.concatenate((gascc,gasccyear))

    hydroyear = dat.loc[dat['Fuel']=="Hydro"]
    hydroyear = hydroyear[hydroyear.columns[4::2]].to_numpy() + hydroyear[hydroyear.columns[5::2]].to_numpy()
    hydroyear = np.reshape(hydroyear,np.prod(hydroyear.shape))
    hydro = np.concatenate((hydro,hydroyear))
    
    nuclearyear = dat.loc[dat['Fuel']=="Nuclear"]
    nuclearyear = nuclearyear[nuclearyear.columns[4::2]].to_numpy() + nuclearyear[nuclearyear.columns[5::2]].to_numpy()
    nuclearyear = np.reshape(nuclearyear,np.prod(nuclearyear.shape))
    nuclear = np.concatenate((nuclear,nuclearyear))
    
    otheryear = dat.loc[dat['Fuel']=="Other"]
    otheryear = otheryear[otheryear.columns[4::2]].to_numpy() + otheryear[otheryear.columns[5::2]].to_numpy()
    otheryear = np.reshape(otheryear,np.prod(otheryear.shape))
    other = np.concatenate((other,otheryear))

    solaryear = dat.loc[dat['Fuel']=="Solar"]
    solaryear = solaryear[solaryear.columns[4::2]].to_numpy() + solaryear[solaryear.columns[5::2]].to_numpy()
    solaryear = np.reshape(solaryear,np.prod(solaryear.shape))
    solar = np.concatenate((solar,solaryear))

    windyear = dat.loc[dat['Fuel']=="Wind"]
    windyear = windyear[windyear.columns[4::2]].to_numpy() + windyear[windyear.columns[5::2]].to_numpy()
    windyear = np.reshape(windyear,np.prod(windyear.shape))
    wind = np.concatenate((wind,windyear))
    
biomass = np.nan_to_num(biomass)
coal = np.nan_to_num(coal) # / (18774/2) # (14297/2)
gas = np.nan_to_num(gas) # / (47259/2)
gascc = np.nan_to_num(gascc)
hydro = np.nan_to_num(hydro)
nuclear = np.nan_to_num(nuclear)
other = np.nan_to_num(other)
solar = np.nan_to_num(solar)
wind = np.nan_to_num(wind) # / (9452/2)
solar = np.nan_to_num(solar)
#solar = pd.read_excel('SolarProfile2011.xlsx',usecols=['Total']).to_numpy().flatten()
#solar = solar / max(solar)
#oldrenewables = wind+other+hydro
#total = coal+gas+nuclear+oldrenewables
oldrenewables = solar+wind+other+biomass+hydro
total = coal+gas+gascc+nuclear+oldrenewables
total3 = np.hstack((total,total,total))

pdat = pd.read_csv('Power90.csv')
solarp = pdat['Solar'].to_numpy()
windp = pdat['Wind'].to_numpy()
'''
solarpower = np.empty(17520)
windpower = np.empty(17520)
for i in range(17520):
    solarpower[i] = sum(solarp[i::17520])/3
    windpower[i] = sum(windp[i::17520])/3
    
pdat = pd.read_excel('Sites90.xlsx',sheet_name='SitesLong')
solarout = pdat['Solar'].dropna().to_list()
windout = pdat['Wind'].dropna().to_list()
sitesout = pdat['Solar'].dropna().to_list() + pdat['Wind'].to_list()

solarmodel = np.zeros(52560)
windmodel = np.zeros(52560)
sc = 0; wc = 0;

for site in solarout:
    i = solarnames.index(site)
    solarmodel += solarpower[:,i]
    cap = float(site.split(': ')[1])
    sc += cap
for site in windout:
    i = windnames.index(site)
    windmodel += windpower[:,i]
    cap = float(site.split(': ')[1])
    wc += cap

totalmodel = 2*(solarmodel + windmodel) / (sc+wc)
solarmodel = 2*solarmodel / sc
windmodel = 2*windmodel / wc

gas3 = 2*(gas + gascc) / (19310 + 35564)
gas3 = np.hstack((gas3,gas3,gas3))

solarmodelh = (solarmodel[::2] + solarmodel[1::2]) / 2
windmodelh = (windmodel[::2] + windmodel[1::2]) / 2
windh = (wind[::2] + wind[1::2]) / 2
coalh = (coal[::2] + coal[1::2]) / 2
gash = (gas[::2] + gas[1::2]) / 2
print(coalh.shape)
'''

load2009 = pd.read_excel('ERCOT/Load/2009_ERCOT_Hourly_Load_Data.xls',usecols=['ERCOT']).to_numpy().flatten()
load2009 = np.repeat(load2009, 2)
load2010 = pd.read_excel('ERCOT/Load/2010_ERCOT_Hourly_Load_Data.xls',usecols=['ERCOT']).to_numpy().flatten()
load2010 = np.repeat(load2010, 2)
load2011 = pd.read_excel('ERCOT/Load/Native_Load_2011.xls',usecols=['ERCOT']).to_numpy().flatten()
load2011 = np.repeat(load2011, 2)
load = pd.read_excel('ERCOT/Load/Native_Load_2019.xlsx',usecols=['ERCOT']).to_numpy().flatten()
load2019 = np.repeat(load, 2)

y = 17520
gas2019 = gas + gascc
coal2019 = coal
outdic = {'Load 2009': load2009,'Load 2010': load2010,'Load 2011': load2011, 'Load 2019': load2019, 'Solar 2009': 2*solarp[0:y],
          'Solar 2010': 2*solarp[y:(2*y)], 'Solar 2011': 2*solarp[(2*y):(3*y)], 'Wind 2009': 2*windp[0:y],
          'Wind 2010': 2*windp[y:(2*y)], 'Wind 2011': 2*windp[(2*y):(3*y)], 'Coal 2019': 2*coal2019, 'Gas 2019': 2*gas2019}
# pd.DataFrame(data = outdic).to_csv('Output_vs_Load.csv')



load3 = np.hstack((load,load,load))

#oldrenewablesh = oldrenewables[::2] + oldrenewables[1::2]
coalh = coal[::2] + coal[1::2]
solarph = solarp[::2] + solarp[1::2]
windph = windp[::2] + windp[1::2]

#oldrenew3 = np.hstack((oldrenewablesh,oldrenewablesh,oldrenewablesh))
coalh3 = np.hstack((coalh,coalh,coalh))
coal3 = np.hstack((coal,coal,coal))
old = load3 - coalh3
new = load3 - (solarph+windph)


allgas = gas + gascc
gas3 = np.hstack((allgas,allgas,allgas))
dif = coal3 - (solarp+windp)
newgas = gas3+dif

In [ ]:
#Figure 11 - fuel mix graph
bh = np.empty(48); ch = np.empty(48); gh = np.empty(48); gcch = np.empty(48); hh = np.empty(48); nh = np.empty(48);
oh = np.empty(48); sh = np.empty(48); wh = np.empty(48); sph = np.empty(48); wph = np.empty(48);

mi = 8688 #0 #0
mf = 10176 #1488 #17520
dn = 31 #31 #365 

for h in range(48):
    bh[h] = (2*sum(biomass[mi+h:mf:48])) / dn
    ch[h] = (2*sum(coal[mi+h:mf:48])) / dn
    gh[h] = (2*sum(gas[mi+h:mf:48])) / dn
    gcch[h] = (2*sum(gascc[mi+h:mf:48])) / dn
    hh[h] = (2*sum(hydro[mi+h:mf:48])) / dn
    nh[h] = (2*sum(nuclear[mi+h:mf:48])) / dn
    oh[h] = (2*sum(other[mi+h:mf:48])) / dn
    sh[h] = (2*sum(solar[mi+h:mf:48])) / dn
    wh[h] = (2*sum(wind[mi+h:mf:48])) / dn
    sph[h] = (2*sum(solarpower[mi+h:mf:48])) / dn
    wph[h] = (2*sum(windpower[mi+h:mf:48])) / dn
    
#formatting to make that graph I want (basically need to put things on top of one another)
total = nh+hh+bh+oh+sh+wh+gh+gcch+ch
    
outdic = {'Nuclear':nh, 'Hydrothermal/Biomass/Other':hh+bh+oh, 'Solar':sh, 'New Solar':sph, 'Wind':wh, 'New Wind':wph, 'Gas-CC':gcch, 'Gas':gh, 'Coal':ch, 'Total ERCOT Generation':total}

pd.DataFrame(data=outdic).to_excel('FuelHoursJul.xlsx') 

In [ ]:
#Figure 12 - Required Ramping of other energy sources to satisfy load (based off fuel mix and model data)
yh = 8760
oldavg = sum(old) / (yh*3)
newavg = sum(new)/ (yh*3)
oldpeak = max(old)
newpeak = max(new)
oldstd = np.std(old)
newstd = np.std(new)
print(oldpeak/oldavg,oldstd,newpeak/newavg,newstd)

#monthly comparison of old and new required ramping
#can easily be done for full year if mi = 0, mf = yh, and days[month] is set to 365
outday = {}
for month in range(12):
    oldday = np.empty(24)
    newday = np.empty(24)
    mi = hsums[month] // 2 #divide by 2 because half-hours, no rounding errors in division (all will be integers)
    mf = hsums[month+1] // 2
    for h in range(24):
        oldday[h] = sum(old[mi+h:mf:24] + old[mi+h+yh:mf+yh:24] + old[mi+h+(2*yh):mf+(2*yh):24]) / (days[month]*3)
        newday[h] = sum(new[mi+h:mf:24] + new[mi+h+yh:mf+yh:24] + new[mi+h+(2*yh):mf+(2*yh):24]) / (days[month]*3)
        outday['Old' + months[month]] = oldday
        outday['New' + months[month]] = newday

#top level analysis
olddist = np.flip(np.sort(old / oldavg))
newdist = np.flip(np.sort(new / newavg))
outlong = {'Old Req':old, 'New Req': new}
outdist = {'Old Req Dist':olddist, 'New Req Dist': newdist}

#pd.DataFrame(data=outlong).to_excel('RequiredRamping.xlsx')
pd.DataFrame(data=outday).to_excel('RequiredRamping.xlsx')

#plt.plot(new, color='blue')
#plt.plot(old, color='orange')

In [ ]:
#model info and analysis of slack
surh = dif < 0
surplus = abs(dif[surh])
slkh = dif > 0
slack = dif[slkh]
old1 = sum(surh)
old2 = sum(surplus)
old3 = sum(slkh)
old4 = sum(slack)

surlh = newgas < 0
surloss = abs(newgas[surlh]) / 2
slklh = newgas > gc3
slackloss = newgas - gc3; slackloss = slackloss[slklh] / 2
new1 = sum(surlh)
new2 = sum(surloss)
new3 = sum(slklh)
new4 = sum(slackloss)
print(old2,old4,new2,new4)
print(new1/old1,new2/old2,new3/old3,new4/old4)
print(new3 / 52560, new4 / sum(coal3))
print(new4 / new3)
#1.9% of surplus will need to be curtailed (or replace other things), on 4.1% of all half-hours where the model produces a surplus
#0.03% of coal production will be unable to be met by spare gas. These losses will occur on 0.1% of half-hours
#the average loss will be 500 MWh

In [ ]:
#Slack by Month
slack = pd.read_csv('Slack90.csv',usecols=['Slack']).to_numpy().flatten()

months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
daysog = [31,28,31,30,31,30,31,31,30,31,30,31]
daysleap = [31,29,31,30,31,30,31,31,30,31,30,31]
t = 17520
yrhours = [t,t,t] #for leap this would be t + 48
baseyear = 2009

ms = np.zeros(12)
cms = np.zeros(12)
ys = np.zeros(3)

for halfhour in range(len(slack)):
    oghh = halfhour
    yi = 0
    while halfhour >= yrhours[yi]:
        halfhour -= yrhours[yi]
        yi += 1
    
    year = baseyear + yi
    
    if (year % 4) == 0:
        days = daysleap
    else:
        days = daysog

    day = (halfhour // 48) + 1
    mi = 0
    while day > days[mi]:
        day -= days[mi]
        mi += 1  
    
    month = months[mi]
    hour = (halfhour % 48) / 2
    ys[yi] += slack[oghh]
    ms[mi] += slack[oghh]
    
for halfhour in range(t):
    oghh = halfhour

    days = daysog
    day = (halfhour // 48) + 1
    mi = 0
    while day > days[mi]:
        day -= days[mi]
        mi += 1  
    
    month = months[mi]
    cms[mi] += coalhours[oghh]

# pd.DataFrame(data={'Slack':ms}).to_csv('SlackMonths.csv')   
# pd.DataFrame(data={'Slack':ys}).to_csv('SlackYears.csv')
# pd.DataFrame(data={'Coal':cms}).to_csv('CoalMonths.csv')

In [ ]:
#Figure 13 - regional power sums of coal and model
coalregs = pd.read_csv('CoalRegional2019.csv')
regsums = {}
totals = [0,0,0]
for reg in regions:
    regsums[reg] = [sum(solarregout[reg])/(3*1000),sum(windregout[reg])/(3*1000),sum(coalregs[reg])/1000]
    totals[0] += sum(solarregout[reg])/(3*1000); totals[1] += sum(windregout[reg])/(3*1000)
    totals[2] += sum(coalregs[reg])/1000
revd = 0
powerall = np.zeros(17520)
for reg in loadzones:
    regpower = solarloadout[reg]+windloadout[reg]
    revd += np.matmul(energyprice[reg],regpower)
    print(sum(regpower) / 1000)
    powerall += regpower
rev = np.matmul(energyprice['HB_HUBAVG'],powerall)
revcoal = np.matmul(energyprice['HB_HUBAVG'],coalhours[0:17520])
scale = 10**9
print(sum(coalhours[0:17520]) / 1000)
print(totals)
#print(revd / scale, rev / scale, revcoal / scale)
#print(regsums)
pd.DataFrame(data=regsums).to_csv('RegSums.csv')

In [ ]:
#Quartiles - NOT USED
csv = 'ModelOut/Power90.csv'
pdat = pd.read_csv(csv)
allpower = 2*(pdat['Solar'].to_numpy() + pdat['Wind'].to_numpy())
print(allpower.shape)
n = 17520

year1 = allpower[0:n].flatten()
year2 = allpower[n:(2*n)].flatten()
year3 = allpower[(2*n):(3*n)].flatten()

day1 = np.empty(48)
day2 = np.empty(48)
day3 = np.empty(48)
jan1 = np.empty(48)
jan2 = np.empty(48)
jan3 = np.empty(48)
jul1 = np.empty(48)
jul2 = np.empty(48)
jul3 = np.empty(48)
#aprall = np.empty(48)
#octall = np.empty(48)

for i in range(48):
    day = np.sort(allpower[i::48])   #(year1[i::48])
    day1[i] = (day[273]) #1st quratile ie median of first half (day[90] + day[91])/2
    day2[i] = day[547] #2nd quartile ie median (day[182])
    day3[i] = day[821]# (day[273] + day[274]) / 2
    jan = np.sort(np.hstack((year1[i:1488:48],year2[i:1488:48],year3[i:1488:48])))   #(year1[i:1488:48])
    jan1[i] = (jan[22] + jan[23]) / 2 #jan[7]
    jan2[i] = jan[46]#jan[15]
    jan3[i] = (jan[69] + jan[70]) / 2 #jan[23]
    jul = np.sort(np.hstack((year1[8688+i:10176:48],year2[8688+i:10176:48],year3[8688+i:10176:48])))   #(year1[8688+i:10176:48])
    jul1[i] = (jul[22] + jul[23]) / 2 #jul[7]
    jul2[i] = jul[46] #jul[15]
    jul3[i] = (jul[69] + jul[70]) / 2 #jul[23]
    #aprall[i] = (sum(year1[4320+i:5760:48]) + sum(year2[4320+i:5760:48]) + sum(year3[4320+i:5760:48])) / (3*30)
    #octall[i] = (sum(year1[13104+i:14592:48]) + sum(year2[13104+i:14592:48]) + sum(year3[13104+i:14592:48])) / (3*31)

#years = {'2009 Year': year1, '2010 Year': year2, '2011 Year': year3}
#alld = {'2009 Day': day1, '2010 Day': day2, '2011 Day': day3}
alld = {'Year Q1': day1, 'Year Q2': day2, 'Year Q3': day3}
alld['Jan Q1'] = jan1
alld['Jan Q2'] = jan2
alld['Jan Q3'] = jan3
alld['Jul Q1'] = jul1 
alld['Jul Q2'] = jul2
alld['Jul Q3'] = jul3
#aproct = {'April': aprall, 'October': octall}

#pd.DataFrame(data=alld).to_excel('Model90Qs.xlsx')